In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
df = pd.read_csv("../UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [3]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [4]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [5]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [6]:
df_alt = df.drop(columns = ["SUPERFICIE", "PRODUCCION"])
X_df_alt = df_alt.drop(columns = ["ALTITUD", "CAMPAÑA"])
y_df_alt = df_alt["ALTITUD"]

X_df_alt_train = X_df_alt[df["ALTITUD"].isna() == False]
y_df_alt_train = y_df_alt[df["ALTITUD"].isna() == False]

X_to_predict = X_df_alt[df["ALTITUD"].isna() == True]

In [7]:
y_df_alt_train

0           660
1           660
2           520
3           520
12      650-660
         ...   
9596        700
9597        700
9598        700
9599        700
9600        700
Name: ALTITUD, Length: 9547, dtype: object

In [8]:
X_df_alt_train_n390 = X_df_alt_train[y_df_alt_train != "390"]
y_df_alt_train_n390 = y_df_alt_train[y_df_alt_train != "390"]

In [9]:
y_df_alt_train_n390_enc = pd.Series(LabelEncoder().fit_transform(y_df_alt_train_n390))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBClassifier(),
                param_grid={"n_estimators": [300, 500, 1000],
                            "learning_rate": [0.15],
                            "max_depth": [40, 60, 100],
                            "min_child_weight": [1, 2],
                            "gamma":[0.0],
                            "colsample_bytree":[0.4],},
                cv=2, scoring=["r2", "neg_root_mean_squared_error"], refit = "r2", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    return gsc

In [26]:
alt_grid = grid(X_df_alt_train_n390, y_df_alt_train_n390_enc)

In [10]:
#params_model = alt_grid.best_params_
params_model={"n_estimators": 300,
              "learning_rate": 0.15,
              "max_depth": 40,
              "min_child_weight": 1,
              "gamma":0.0,
              "colsample_bytree":0.4}

In [11]:
print(params_model)

{'n_estimators': 300, 'learning_rate': 0.15, 'max_depth': 40, 'min_child_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.4}


In [12]:
label_enc = LabelEncoder().fit(y_df_alt_train)
y_df_alt_train_enc = pd.Series(label_enc.transform(y_df_alt_train))

In [13]:
import xgboost as xgb
model_alt_grid = xgb.XGBClassifier(**params_model)
model_alt_grid.fit(X_df_alt_train,y_df_alt_train_enc)
predicted = model_alt_grid.predict(X_to_predict)

In [14]:
df2 = pd.read_csv("../UH_2023/UH_2023_TRAIN.txt", sep = "|")
df2 = df2.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [15]:
df2

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [16]:
predicted2 = pd.Series(label_enc.inverse_transform(predicted))
predicted2.index = df2[df2["ALTITUD"].isna() == True].index

In [17]:
df2.fillna({"ALTITUD": predicted2}, inplace=True)

In [18]:
df2

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,650-660,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [19]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df2["ALTITUD_MIN"] = df2["ALTITUD"].apply(fun, args=("min",))
df2["ALTITUD_DIF"] = df2["ALTITUD"].apply(fun, args=("diff",))
df2["ALTITUD"] = df2["ALTITUD"].apply(fun, args=("nan_1",))

In [20]:
df2.loc[df["MODO"]=="1", "MODO"]  = "0"
df2.loc[df["MODO"]=="2", "MODO"]  = "1"

In [21]:
df_new = df2

#### Splitting dataset

#### 1. cas genèric
- TRAIN: 16-21 
- TEST: 22 gairebé tot

#### 2. sense id_finca amb superfície
- TRAIN: 20-21
- TEST: 22 - gairebé totes les finques que no apareixen el 16-21

#### 3. sense id_finca ni id_zona amb superfície
- TRAIN: 20-21
- TEST: 22 - finques, zones que no apareixen el 16-21 --> zona 233

#### 4. amb producció predita
- TRAIN: 14 + 22 predit
- TEST: 22 - finques, zones que no apareixen el 16-21 --> finca 13450



In [103]:
df_new["CAMPAÑA_ESTACION"] = "20" + df_new["CAMPAÑA"].astype(str)+ "_" + df_new["ID_ESTACION"].astype(str)

#### 16-21 dataset

In [39]:
df_new_14_15 = df_new[df_new["CAMPAÑA"].isin(["14", "15"]) == False]

In [42]:
df_new_14_15.to_csv("../DATASETS_FINALS/TOT_menys_14_15.csv", index=False)

#### 20-21 dataset

In [70]:
df_new_20_21 = df_new[df_new["CAMPAÑA"].isin(["20", "21"]) == True]

In [71]:
df_new_20_21 = df_new_20_21[df_new_20_21["SUPERFICIE"] != 0]

In [78]:
df_new_20_21.to_csv("../DATASETS_FINALS/TOT_20_21.csv", index=False)

#### 14 i 22 dataset

In [104]:
df_new_14_22 = df_new[df_new["CAMPAÑA"].isin(["14", "22"]) == True]

In [105]:
df_new_14_22.to_csv("../DATASETS_FINALS/TOT_14_22.csv", index_label = "INDEX")

#### Intersections

In [93]:
df_new5 = df_new.copy()
df_16_21 = df_new[df_new["CAMPAÑA"].isin(["14", "15", "22"]) == False]
df_20_21 = df_new[df_new["CAMPAÑA"].isin(["20", "21"]) == True]
df_22 = df_new[df_new["CAMPAÑA"].isin(["22"]) == True]
id_finca = set(df_22["ID_FINCA"]) - set(df_16_21["ID_FINCA"])
id_zona = set(df_22["ID_ZONA"]) - set(df_16_21["ID_ZONA"])
#id_estacion = set(df_22["ID_ESTACION"]) - set(df_16_21["ID_ESTACION"])
#id_color = set(df_22["COLOR"]) - set(df_16_21["COLOR"])
#id_tipo = set(df_22["TIPO"]) - set(df_16_21["TIPO"])
#id_modo = set(df_22["MODO"]) - set(df_16_21["MODO"])

In [ ]:
#FINCA 13450 apareix l'any 14
df_new5[df_new5["ID_FINCA"].isin(id_finca) == True]

In [ ]:
df_new5[df_new5["ID_ZONA"].isin(id_zona) == True]

In [ ]:
#ZONA 233 NO APAREIX 14-21
set(df_22[df_22["ID_FINCA"].isin(id_finca) == True]["ID_ZONA"])- set(df_20_21["ID_ZONA"])

#### Cas 1

In [73]:
df_22_cas1 = df_22[df_22["ID_FINCA"].isin(id_finca) == False]

In [74]:
df_22_cas1.to_csv("../DATASETS_FINALS/df_22_cas1.csv", index_label = "INDEX")

#### Cas 2

In [94]:
df_22_cas2 = df_22.copy()

In [95]:
df_22_cas2 = df_22_cas2[(df_22_cas2["ID_FINCA"].isin(id_finca) == True) &
                             (df_22_cas2["ID_FINCA"] != "13450") &
                             (df_22_cas2["ID_ZONA"] != "233")]

In [96]:
df_22_cas2 = df_22_cas2.drop(columns = ["ID_FINCA"])

In [98]:
df_22_cas2.to_csv("../DATASETS_FINALS/df_22_cas2.csv", index_label = "INDEX")

#### Cas 3

In [99]:
df_22_cas3 = df_22.copy()

In [100]:
df_22_cas3 = df_22_cas3[df_22_cas3["ID_ZONA"] == "233"]

In [101]:
df_22_cas3 = df_22_cas3.drop(columns = ["ID_FINCA", "ID_ZONA"])

In [102]:
df_22_cas3.to_csv("../DATASETS_FINALS/df_22_cas3.csv", index_label = "INDEX")

#### Cas 4

In [84]:
df_22_cas4 = df_22.copy()

In [85]:
df_22_cas4 = df_22_cas4[df_22_cas4["ID_FINCA"] == "13450"]

In [86]:
df_22_cas4.to_csv("../DATASETS_FINALS/df_22_cas4.csv", index_label = "INDEX")